This notebook is part of https://github.com/AudioSceneDescriptionFormat/splines, see also https://splines.readthedocs.io/.

[back to overview](hermite.ipynb)

# Uniform Cubic Hermite Splines

We derive the basis matrix as well as the basis polynomials
for cubic (= degree 3) Hermite splines.
The derivations for other degrees is left as an exercise for the reader.

In this notebook,
we consider *uniform* spline segments,
i.e. the parameter in each segment varies from $0$ to $1$.
The derivation for *non-uniform* cubic Hermite splines
can be found in [a separate notebook](hermite-non-uniform.ipynb).

In [ ]:
import sympy as sp
sp.init_printing(order='rev-lex')

We load a few tools from [utility.py](utility.py):

In [ ]:
from utility import NamedExpression, NamedMatrix

In [ ]:
t = sp.symbols('t')

We are considering a single cubic polynomial segment of a Hermite spline
(which is sometimes called *Ferguson cubic*).
This polynomial has 4 coefficients, $\boldsymbol{a}$ to $\boldsymbol{d}$.

In [ ]:
coefficients = sp.Matrix(sp.symbols('a:dbm')[::-1])
coefficients

Combined with the *monomial basis* ...

In [ ]:
b_monomial = sp.Matrix([t**3, t**2, t, 1]).T
b_monomial

... the coefficients form an expression
for our polynomial segment $\boldsymbol{p}(t)$,
where $0 \le t \le 1$:

In [ ]:
p = NamedExpression('pbm', b_monomial.dot(coefficients))
p

For more information about polynomials,
see [Polynomial Parametric Curves](polynomials.ipynb).

Let's also calculate the first derivative
(a.k.a. velocity, a.k.a. tangent vector),
while we are at it:

In [ ]:
pd = p.diff(t)
pd

To generate a Hermite spline segment,
we have to provide the value of the polynomial
at time $t_0 = 0$ and $t_1 = 1$
(the start and end point of the segment).
We also have to provide the first derivative at the start and at the end.

\begin{align*}
\boldsymbol{x}_0 &= \left.\boldsymbol{p}\right\rvert_{t=0}\\
\boldsymbol{x}_1 &= \left.\boldsymbol{p}\right\rvert_{t=1}\\
\boldsymbol{\dot{x}}_0 &= \left.\frac{d}{dt}\boldsymbol{p}\right\rvert_{t=0}\\
\boldsymbol{\dot{x}}_1 &= \left.\frac{d}{dt}\boldsymbol{p}\right\rvert_{t=1}
\end{align*}

We call those 4 values the *control values* of the segment.

Evaluating the polynomial and its derivative
at times $0$ and $1$ leads to 4 expressions for our 4 control values:

In [ ]:
x0 = p.evaluated_at(t, 0).with_name('xbm0')
x1 = p.evaluated_at(t, 1).with_name('xbm1')
xd0 = pd.evaluated_at(t, 0).with_name('xdotbm0')
xd1 = pd.evaluated_at(t, 1).with_name('xdotbm1')

In [ ]:
display(x0, x1, xd0, xd1)

Given an input vector of control values ...

In [ ]:
control_values_H = NamedMatrix(sp.Matrix([x0.name,
                                          x1.name,
                                          xd0.name,
                                          xd1.name]))
control_values_H.name

... we want to find a way to transform those into the coefficients of our cubic polynomial.

In [ ]:
M_H = NamedMatrix(r'{M_\text{H}}', 4, 4)

In [ ]:
coefficients_H = NamedMatrix(coefficients, M_H.name * control_values_H.name)
coefficients_H

This way, we can express our previously unknown coefficients
in terms of the given control values.

However, in order to make it easy to determine
the coefficients of the *basis matrix* $M_H$,
we need the equation the other way around
(by left-multiplying by the inverse):

In [ ]:
control_values_H.expr = M_H.name.I * coefficients
control_values_H

We can now insert the expressions for the control values
that we obtained above ...

In [ ]:
substitutions = x0, x1, xd0, xd1

In [ ]:
control_values_H.subs_symbols(*substitutions)

... and from this equation we can directly read off
the matrix coefficients of ${M_H}^{-1}$:

In [ ]:
M_H.I = sp.Matrix(
    [[expr.coeff(cv) for cv in coefficients]
     for expr in control_values_H.subs_symbols(*substitutions).name])
M_H.I

The same thing for copy & paste purposes:

In [ ]:
print(_.expr)

This transforms the coefficients of the polynomial into our control values,
but we need it the other way round,
which we can simply get by inverting the matrix:

In [ ]:
M_H

Again, for copy & paste:

In [ ]:
print(_.expr)

Multiplying the monomial basis with this matrix
yields the *Hermite basis polynomials*:

In [ ]:
b_H = NamedMatrix(r'{b_\text{H}}', b_monomial * M_H.expr)
b_H.factor().simplify().T

In [ ]:
from helper import plot_basis

In [ ]:
plot_basis(*b_H.expr, labels=control_values_H.name)

Note that the basis function associated with $\boldsymbol{x}_0$
has the value $1$ at the beginning,
while all others are $0$ at that point.
After that, the influence of the basis function
associated with $\dot{\boldsymbol{x}}_0$
is growing rapidly.
Towards the end, both functions must relinquish their influence
to the other two basis functions.

To quickly check whether the matrix $M_H$ does what we expect,
let's plot an example segment:

In [ ]:
import numpy as np

If we use the same API as for the other splines,
we can reuse the helper functions for plotting
from [helper.py](helper.py).

In [ ]:
from helper import plot_spline_2d, plot_tangents_2d

In [ ]:
class UniformHermiteSegment:

    grid = 0, 1

    def __init__(self, control_values):
        self.coeffs = sp.lambdify([], M_H.expr)() @ control_values

    def evaluate(self, t):
        t = np.expand_dims(t, -1)
        return t**[3, 2, 1, 0] @ self.coeffs

<div class="alert alert-info">

Note

The `@` operator is used here to do
[NumPy's matrix multiplication](https://numpy.org/doc/stable/reference/generated/numpy.matmul.html).

</div>

In [ ]:
vertices = [0, 0], [5, 1]
tangents = [2, 3], [0, -2]

In [ ]:
s = UniformHermiteSegment([*vertices, *tangents])

In [ ]:
plot_spline_2d(s, chords=False)
plot_tangents_2d(tangents, vertices)

## Relation to Bézier Splines

Above, we were using two positions (start and end)
and two tangent vectors (at those same two positions) as control values:

In [ ]:
control_values_H.name

What about using four positions (and no tangent vectors) instead?

Let's use the point $\boldsymbol{\tilde{x}}_0$ as a "handle"
(connected to $\boldsymbol{x}_0$) that controls the tangent vector.
Same for $\boldsymbol{\tilde{x}}_1$ (connected to $\boldsymbol{x}_1$).

And since the tangents looked unwieldily long in the plot above
(compared to the effect they have on the shape of the curve),
let's put the handles only at a third of the length of the tangents,
shall we?

\begin{align*}
\tilde{\boldsymbol{x}}_0
&=
\boldsymbol{x}_0 + \frac{\dot{\boldsymbol{x}}_0}{3}
\\
\tilde{\boldsymbol{x}}_1
&=
\boldsymbol{x}_1 - \frac{\dot{\boldsymbol{x}}_1}{3}
\end{align*}

In [ ]:
control_values_B = NamedMatrix(sp.Matrix([
    x0.name,
    sp.Symbol('xtildebm0'),
    sp.Symbol('xtildebm1'),
    x1.name,
]), sp.Matrix([
    x0.name,
    x0.name + xd0.name / 3,
    x1.name - xd1.name / 3,
    x1.name,
]))
control_values_B

Now let's try to come up with a matrix
that transforms our good old Hermite control values
into our new control points.

In [ ]:
M_HtoB = NamedMatrix(r'{M_\text{H$\to$B}}', 4, 4)

In [ ]:
NamedMatrix(control_values_B.name, M_HtoB.name * control_values_H.name)

We can immediately read the matrix coefficients
off the previous expression.

In [ ]:
M_HtoB.expr = sp.Matrix([[expr.coeff(cv) for cv in control_values_H.name]
                         for expr in control_values_B.expr])
M_HtoB.pull_out(sp.S.One / 3)

In [ ]:
print(_.expr)

The inverse of this matrix transforms our new control points
into Hermite control values:

In [ ]:
M_BtoH = NamedMatrix(r'{M_\text{B$\to$H}}', M_HtoB.I.expr)
M_BtoH

In [ ]:
print(_.expr)

When we combine $M_H$ with this new matrix,
we get a matrix which leads us to a new set of basis polynomials
associated with the 4 control points.

In [ ]:
M_B = NamedMatrix(r'{M_\text{B}}', M_H.name * M_BtoH.name)
M_B

In [ ]:
M_B = M_B.subs_symbols(M_H, M_BtoH).doit()
M_B

In [ ]:
b_B = NamedMatrix(r'{b_\text{B}}', b_monomial * M_B.expr)
b_B.T

In [ ]:
plot_basis(*b_B.expr, labels=control_values_B.name)

Those happen to be the cubic *Bernstein* polynomials and
it turns out that we just invented *Bézier* curves!
See [the section about Bézier/Bernstein splines](bezier.ipynb)
for more information about them.

We chose the additional control points to be located
at $\frac{1}{3}$ of the tangent vector.
Let's quickly visualize this
using the example from above and $M_\text{H$\to$B}$:

In [ ]:
points = sp.lambdify([], M_HtoB.expr)() @ [*vertices, *tangents]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plot_spline_2d(s, chords=False)
plot_tangents_2d(tangents, vertices)
plt.scatter(*points.T, marker='X', color='black')
plt.annotate(r'$\quad\tilde{\bf{x}}_0$', points[1])
plt.annotate(r'$\tilde{\bf{x}}_1\quad$', points[2], ha='right');